In [123]:
import pandas as pd
import pickle
import re
import string
import random
import numpy as np
import tqdm

random.seed(3)

In [ ]:
read

In [125]:
df.head()

,index,artist_all,artist_base,rank,song,year,artist_featured,song_clean,artist_clean,lyrics,source
0,0,percy faith,percy faith,1,theme from a summer place,1960,NaN,theme from a summer place,percy faith,theres a summer place where it may rain or sto...,genius.com
1,1,jim reeves,jim reeves,2,he'll have to go,1960,NaN,hell have to go,jim reeves,put your sweet lips a little closer to the pho...,songlyrics.com
2,2,the everly brothers,the everly brothers,3,cathy's clown,1960,NaN,cathys clown,the everly brothers,dont want your love any more dont want your k...,genius.com
3,3,johnny preston,johnny preston,4,running bear,1960,NaN,running bear,johnny preston,on the bank of the river stood running bear yo...,genius.com
4,4,mark dinning,mark dinning,5,teen angel,1960,NaN,teen angel,mark dinning,teen angel teen angel teen angel that fateful ...,genius.com


In [126]:
lyrics = df["lyrics"]
lyrics = lyrics[~lyrics.isna()].str.strip()
lyrics = lyrics.sample(n=400, random_state=33)
lyrics = lyrics.tolist()
len(lyrics)

400

In [127]:
lyrics

['little things that you do make me glad im in love with you little things that you say make me glad that i feel this way the way you smile the way you hold my hand and when im down you always understand you know i love those little things in my ear that you say when theres no one near little things that you do let me know that your love is true when we walk you like to hold my hand and when we talk you tell me im your man you know i love those little things that i hear the little things you whisper in my ear i know there aint nobody else like you no one could do the little things you do oo little things that you do make me glad im in love with you little things that you say make me glad that i feel this way when we walk you like to hold my hand and when we talk you tell me im your man you know i love those little things that i hear the little things you whisper in my ear i know there aint nobody else like you no one could do the little things you do oo',
 'i see the bad moon arising i

In [128]:
def clean_text(s):
    regex = re.compile("[\(\[].*?[\)\]]")
    result = re.sub(regex, "", s)
    result = result.replace("\n", " ").strip().lower()
    result = re.sub(' +', ' ', result)
    result = result.translate(str.maketrans('', '', string.punctuation))
    return result

In [129]:
with open("data/dataset.pkl", "rb") as fd:
    beatles = pickle.load(fd)
beatles = list(map(lambda x: clean_text(x["lyrics"]), beatles))
beatles

['1 2 3 4 well she was just 17 if you know what i mean and the way she looked was way beyond compare so how could i dance with another when i saw her standing there well she looked at me and i i could see that before too long id fall in love with her she wouldnt dance with another when i saw her standing there well my heart went boom when i crossed that room and i held her hand in mine well we danced through the night and we held each other tight and before too long i fell in love with her now ill never dance with another since i saw her standing there well my heart went boom when i crossed that room and i held her hand in mine oh we danced through the night and we held each other tight and before too long i fell in love with her now ill never dance with another since i saw her standing there oh since i saw her standing there yeah well since i saw her standing there',
 'the world is treating me bad misery im the kind of guy who never used to cry the world is treating me bad misery ive 

In [130]:
with open("data/generated.pkl", "rb") as fd:
    gen = pickle.load(fd)
gen = list(map(lambda x: clean_text(x), gen))
gen 

['woooooh baby some fun tonight well were going to have some fun tonight have some fun tonight everythings alright have some fun tonight yeah well have some fun some fun tonight i been told when a boy kiss a girl take a trip around the world hey hey bop shuop mbop bop shuop hey hey bop shuop mbop bop shuop hey hey bop shuop mbop bop shuop hey hey bop shuop yeah she say you do bop shuop yeah she say you do bop shuop well i talk about boys yeah yeah boys dont ya know i mean boys yeah yeah boys well i talk about boys now yeah i mean you do yeah yeah boys well i talk about boys now yeah yeah boys aaah boys yeah yeah boys well i talk about boys now yeah yeah boys aaah boys yeah yeah boys well i talk about boys now yeah yeah boys aaah boys yeah yeah boys well i talk about boys now',
 'made me cry im sorry that i doubted you i was so unfair you i find you in my car i dont told you my car and maybe ill love you i told that girl i can start right away when she said listen babe i got something t

In [121]:
random.shuffle(beatles)
random.shuffle(lyrics)

In [61]:
x_train = beatles[:-50] + lyrics[:-50]
y_train = ["beatles"] * (len(beatles) - 50) + ["na"] * (len(lyrics) - 50)

x_test = beatles[-50:] + lyrics[-50:]
y_test = ["beatles"] * 50 + ["na"] * 50

In [62]:
x_gen = gen

In [63]:
len(x_train), len(y_train), len(x_test), len(y_test), len(x_gen)

(614, 614, 100, 100, 25)

In [64]:
from nltk.corpus import stopwords
stop = list(stopwords.words('english'))

In [65]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words=set(stop))
X = vectorizer.fit_transform(x_train + x_test + x_gen)

In [66]:
X.shape

(739, 12689)

In [67]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=500, n_iter=7, random_state=42)
X = svd.fit_transform(X)
X.shape

(739, 500)

In [69]:
x_train, x_test, x_gen = X[:-125, :], X[-125:-25, :], X[-25:, :]
x_train.shape, x_test.shape, x_gen.shape

((614, 500), (100, 500), (25, 500))

In [70]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(C=100).fit(x_train, y_train)
clf.score(x_test, y_test)

C:\Users\Gabriel\Miniconda3\envs\ada\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.66

In [71]:
from sklearn.metrics import confusion_matrix
confusion_matrix(clf.predict(x_test), y_test)

array([[27, 11],
       [23, 39]], dtype=int64)

In [72]:
clf.score(x_gen, ["beatles"] * 25)

0.8

In [73]:
confusion_matrix(clf.predict(x_gen), ["beatles"] * 25)

array([[20,  0],
       [ 5,  0]], dtype=int64)

In [131]:
accs = []
vl_accs = []
for i in tqdm.tqdm(range(0, 10)):
    random.shuffle(beatles)
    random.shuffle(lyrics)
    
    x_train = beatles[:-50] + lyrics[:-50]
    y_train = ["beatles"] * (len(beatles) - 50) + ["na"] * (len(lyrics) - 50)

    x_test = beatles[-50:] + lyrics[-50:]
    y_test = ["beatles"] * 50 + ["na"] * 50
    
    x_gen = gen

    vectorizer = TfidfVectorizer(stop_words=set(stop))
    X = vectorizer.fit_transform(x_train + x_test + x_gen)

    svd = TruncatedSVD(n_components=500, n_iter=7, random_state=42)
    X = svd.fit_transform(X)
    
    x_train, x_test, x_gen = X[:-125, :], X[-125:-25, :], X[-25:, :]
    
    clf = LogisticRegression(C=100, random_state=33).fit(x_train, y_train)
    acc = clf.score(x_test, y_test)
    accs.append(acc)
    
    v_acc = clf.score(x_gen, ["beatles"] * 25)
    vl_accs.append(v_acc)


  0%|                                                                                           | 0/10 [00:00<?, ?it/s]C:\Users\Gabriel\Miniconda3\envs\ada\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)

 10%|████████▎                                                                          | 1/10 [00:07<01:05,  7.30s/it]C:\Users\Gabriel\Miniconda3\envs\ada\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)

 20%|████████████████▌                                                                  | 2/10 [00:14<00:57,  7.23s/it]C:\Users\Gabriel\Miniconda3\envs\ada\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  F

In [132]:
np.mean(accs), np.std(accs)

(0.646, 0.01907878402833893)

In [133]:
np.mean(vl_accs), np.std(vl_accs)

(0.8879999999999999, 0.05878775382679626)